# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Описание данных:**

- *children* — количество детей в семье
- *days_employed* — общий трудовой стаж в днях
- *dob_years* — возраст клиента в годах
- *education* — уровень образования клиента
- *education_id* — идентификатор уровня образования
- *family_status* — семейное положение
- *family_status_id* — идентификатор семейного положения
- *gender* — пол клиента
- *income_type* — тип занятости
- *debt* — имел ли задолженность по возврату кредитов
- *total_income* — ежемесячный доход
- *purpose* — цель получения кредита

### Шаг 1. Файл с данными. 

In [1]:
import pandas as pd
from pymystem3 import Mystem

In [26]:
m = Mystem()

data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [27]:
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


### Вывод

Данные содержат ошибки и пропущенные значения:
* столбы days_employed и total_income содержат пропуски;
* также дробное представление данных в этих столбцах излишне, стаж удобнее считать в целых днях, а доход без копеек;
* в столбце days_employed присутствуют аномально большие значения. Сомневаюсь, что типусы могут иметь стаж более 900 лет, они бы уже за 300 лет выстроли свою экономику так, чтобы им больше не пришлось работать;
* также в этом столбце содержится большое количество отрицательных значений;
* в education и family_status следует все значения привести к нижнему регистру.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [28]:
data[data['days_employed'].isna()] #посмотрим строки, содержащие NaN значения

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Гипотеза о том, что данные пропущены у пользователей, которые никогда не работали не подходит, так как значения в столбце income_type свидетельствуют об обратном. Скорее всего при заполнении таблицы часть данных была потеряна.  
Заполним пропуски средними значениями или медианой.  
Для начала избавимся от минусов и аномально больших значений.

In [29]:
data[data['days_employed'] > 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем


In [30]:
data[data['days_employed'] > 0].describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,3445.000000,3445.000000,3445.000000,3445.000000,3445.000000,3445.000000,3445.000000
mean,0.128302,365004.309916,59.124819,0.914659,0.984325,0.052830,137124.105624
std,0.955042,21075.016396,7.580584,0.517103,1.316071,0.223727,80242.210917
min,-1.000000,328728.720605,0.000000,0.000000,0.000000,0.000000,20667.263793
25%,0.000000,346639.413916,56.000000,1.000000,0.000000,0.000000,82876.335652
50%,0.000000,365213.306266,60.000000,1.000000,0.000000,0.000000,118514.486412
75%,0.000000,383246.444219,64.000000,1.000000,2.000000,0.000000,169746.263276
max,20.000000,401755.400475,74.000000,4.000000,4.000000,1.000000,735103.270167


Все положительные значения в *days_employed* являются аномальными.
Также есть клиенты с возрастом 0 лет, таких уберем из данных.

In [31]:
data[data['income_type'] == 'пенсионер'] #хм. пенсионеры какие-то подозрительные, посмотрим на них поближе

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем


In [32]:
print(data[data['income_type'] == 'пенсионер']['income_type'].count()) # количество пенсионеров
data[(data['income_type'] == 'пенсионер') & (data['days_employed'] > 0)]['income_type'].count() # количество пенсионеров с аномально большими значниями стажа

3856


3443

Похоже все строки с "пенсионерами" заполняются некорректно. Причем удаление строк с аномально большими значениями стажа приведет к удалению большей части пенсионеров, тогда заменим их средними значениями.  
Но для начала избавимся от отрицательных значений.

In [33]:
data.loc[data['days_employed'] > 0, 'days_employed'] = 0

In [34]:
data = data[data['dob_years'] > 0]

In [35]:
data.loc[:,['days_employed']] = data['days_employed'] * (-1)
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,-0.000000,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,-0.000000,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [37]:
days_employed_avg = data['days_employed'].mean()
days_employed_avg

1934.7997719135828

In [41]:
import warnings

warnings.filterwarnings('ignore')

In [42]:
data['days_employed'] = data.loc[:,['days_employed']].fillna(days_employed_avg) # заменим отсутствующие значения на среднии
data['days_employed'] = data.loc[:,['days_employed']].replace(0, days_employed_avg) # заменим аномальные значения на среднии
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,1934.799772,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [43]:
# аналогично заменим пропуски в столбце total_income, использую медиану, т.к. высокие зарплаты могут сместить среднее вверх
total_income_median = data['total_income'].median()
data['total_income'] = data['total_income'].fillna(total_income_median)

In [46]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21424.000000,21424.000000,21424.000000,21424.000000,21424.000000,21424.000000,2.142400e+04
mean,0.538275,2243.804705,43.497479,0.817914,0.971294,0.080891,1.651967e+05
std,1.377370,1989.855651,12.246934,0.548224,1.419737,0.272673,9.797611e+04
min,-1.000000,24.000000,19.000000,0.000000,0.000000,0.000000,2.066700e+04
25%,0.000000,1025.000000,33.000000,1.000000,0.000000,0.000000,1.078242e+05
50%,0.000000,1934.000000,43.000000,1.000000,0.000000,0.000000,1.450110e+05
75%,1.000000,2520.000000,53.000000,1.000000,1.000000,0.000000,1.955080e+05
max,20.000000,18388.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Вывод

Пропуски устранены. Выявлено, что столбец days_employed для пенсионеров заполняется некоррекно.

### Замена типа данных

In [45]:
# Заменим тип данных столбцов days_employed и total_income на целочисленный
data[['days_employed', 'total_income']] = data[['days_employed', 'total_income']].astype('int')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,1934,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Вывод

Значения в days_employed и total_income теперь имеют удобный вид

### Обработка дубликатов

In [47]:
data['education'] = data['education'].str.lower() # значения столбца education к нижнему регистру
data['family_status'] = data['family_status'].str.lower() # значения столбца family_status к нижнему регистру

In [48]:
# Так как мы не можем отобрать уникальных пользователей, будем искать дубли по полной строке
data.duplicated().sum()

71

In [49]:
# удалим дубли
data = data.drop_duplicates().reset_index(drop=True)

### Вывод

Дубли обнаружены в незначительном количестве и удалены.

### Лемматизация

In [50]:
data['purpose'].unique() # уникальные цели получения кредита

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [52]:
# функция лемматизирует значение столбца purpose и проверяя по ключевому слову выдает значение нового столбца purpose_category
def purpose_lem(purpose):
    lemmas = m.lemmatize(purpose)
    if 'недвижимость' in lemmas or 'жилье' in lemmas:
        return 'недвижимость'
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'образование' in lemmas:
        return 'образование'
    if 'свадьба' in lemmas:
        return 'свадьба'
    else:
        return 'другое'
    

In [53]:
data['purpose_category'] = data['purpose'].apply(purpose_lem)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,1934,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


In [54]:
data['purpose_category'].unique()

array(['недвижимость', 'автомобиль', 'образование', 'свадьба'],
      dtype=object)

### Вывод

Цели получения кредита поделены на 4 категории.

### Категоризация данных

Выделим два словаря: по семейному положению и по уровню образования.

In [55]:
family_status = data[['family_status_id', 'family_status']]
education_level = data[['education_id', 'education']]

In [56]:
# разделим доход на 4 категории: до 60к от 60к до 100к, от 100к до 200к  и более 200к
def profit_people(total_income):
    if total_income < 60000:
        return 'до 60к'
    elif 60000 <= total_income < 100000:
        return '60-100'
    elif total_income >= 200000:
        return 'более 200к'
    else:
        return 'от 100 до 200'
    

In [57]:
data['profit_cat'] = data['total_income'].apply(profit_people)

### Вывод

Данные категоризированы на два словаря.

### Шаг 3. Исследование

- Проверим, есть ли зависимость между наличием детей и возвратом кредита в срок

In [58]:
# в children значения -1 и 20 - скорее всего опечатки, заменим их на 1 и 2 соответственно
data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)

data_group_children = data.groupby('children').agg({'debt': ['count', 'sum']}).reset_index() # групировка по children с подсчетом суммы невозвратов
data_group_children['ratio'] = data_group_children['debt']['sum'] / data_group_children['debt']['count'] # ratio - индекс невозврата кредита в срок
data_group_children

children   debt           ratio
            count   sum          
0        0  14022  1058  0.075453
1        1   4839   442  0.091341
2        2   2114   202  0.095553
3        3    328    27  0.082317
4        4     41     4  0.097561
5        5      9     0  0.000000

In [59]:
children_pivot = data_group_children.pivot_table(index='children', values='ratio', aggfunc='sum')
children_pivot

children
0    0.075453
1    0.091341
2    0.095553
3    0.082317
4    0.097561
5    0.000000
Name: ratio, dtype: float64

### Вывод

Индекс невозврата для клиентов у которых есть дети при любом количестве детей, за исключением 5 детей (слишком малочисленная группа для исследования) больше чем у клиентов без детей.

- Проверим, есть ли зависимость между семейным положением и возвратом кредита в срок

In [60]:
data_group_family_status = data.groupby('family_status').agg({'debt': ['count', 'sum']}).reset_index() # групировка по family_status с подсчетом суммы невозвратов
data_group_family_status['ratio'] = data_group_family_status['debt']['sum'] / data_group_family_status['debt']['count']
data_group_family_status

family_status   debt          ratio
                          count  sum          
0              в разводе   1185   85  0.071730
1         вдовец / вдова    954   62  0.064990
2       гражданский брак   4130  386  0.093462
3        женат / замужем  12290  927  0.075427
4  не женат / не замужем   2794  273  0.097709

### Вывод

Наибольший результат показывают люди, живущие в гражданском браке и неженатые/незамужнии.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [61]:
data_group_profit = data.groupby('profit_cat').agg({'debt': ['count', 'sum']}).reset_index() # групировка по profit_cat с подсчетом суммы невозвратов
data_group_profit['ratio'] = data_group_profit['debt']['sum'] / data_group_profit['debt']['count']
data_group_profit

profit_cat   debt           ratio
                  count   sum          
0         60-100   3638   304  0.083562
1     более 200к   5040   357  0.070833
2         до 60к    802    49  0.061097
3  от 100 до 200  11873  1023  0.086162

### Вывод

Не у одной категории доходов нет невозвратов более 9%. Наибольшую вероятность невозврата имеют люди, получающие от 100к до 200к. Причины неизвестны.

- Как разные цели кредита влияют на его возврат в срок?

In [62]:
data_group_purpose = data.groupby('purpose_category').agg({'debt': ['count', 'sum']}).reset_index() # групировка по data_group_purpose с подсчетом суммы невозвратов
data_group_purpose['ratio'] = data_group_purpose['debt']['sum'] / data_group_purpose['debt']['count']
data_group_purpose

purpose_category   debt          ratio
                    count  sum          
0       автомобиль   4284  400  0.093371
1     недвижимость  10764  779  0.072371
2      образование   3995  370  0.092616
3          свадьба   2310  184  0.079654

### Вывод

Примерно одинаковые наибольшие индексы невозврата имеют цели для приобретения автомобиля и получение образования. Меньший результат покаывают "недвижимость" и "свадьба", скорее всего люди относятся к покупке недвижимости более серьезно, чем к покупке автомобиля, а кридит за свадьбу могут оплатить подарками гостей.

### Шаг 4. Общий вывод

Индекс невозврата, показывающий для данной категории процент людей не вернувших кредит в срок, принимает наибольшие значения приближенные к 9 %.  
Люди имеющие детей чаще не возвращают кредит в срок, чем бездетные.  
Однако живущие гражданским браком или не состоявшие в браке также имеют высокий индекс невозврата (9,7 % для последних).  
Исследование зависимости между уровнем дохода и возвратом кредита показало, что чаще склонны просрочить кредит люди, получающие ежемесячный доход от 100 до 200 тысяч.
А исследование целей получения кредита, показало, что стоит уделить внимание людям имеющих цели, связанные с автомобилем или образованием.